#### Decision tree for allocating oil type to tanker traffic
see google drawing (Tanker_Oil_Attribution)[https://docs.google.com/drawings/d/1-4gl2yNNWxqXK-IOr4KNZxO-awBC-bNrjRNrt86fykU/edit ] for a visual representation

In [1]:
# These will become function inputs
origin = 'Westridge Marine Terminal'
destination = 'Pacific'
ship_type = 'tanker'
random_seed=None

In [2]:
import numpy 
import yaml
import pathlib

In [3]:
# load the list of CAD, US and generic origins and destinations
master_dir  = '/Users/rmueller/Projects/MIDOSS/analysis-rachael/notebooks/monte_carlo/'
master_file = 'master.yaml'

with open(f'{master_dir}{master_file}') as file:
    master = yaml.safe_load(file)

# Assign US and CAD origin/destinations from master file
CAD_origin_destination = master['categories']['CAD_origin_destination']
US_origin_destination = master['categories']['US_origin_destination']

# Get file paths to fuel-type yaml files
home = pathlib.Path(master['directories'])
CAD_yaml     = home/master['files']['cargo_CAD']
WA_in_yaml   = home/master['files']['cargo_WA_in']
WA_out_yaml  = home/master['files']['cargo_WA_out']
US_yaml      = home/master['files']['cargo_US']
Pacific_yaml = home/master['files']['cargo_Pacific']

#### Create decision-making tree for tanker traffic
All attributions will be tank-cargo at this point.  I don't think we have decision-making around spill type (fuel or cargo) yet (need to check!).  Will likely add here

In [4]:
def get_fuel_type(yaml_file, facility):
     with yaml_file.open("rt") as file:
            cargo = yaml.safe_load(file)
            tanker = cargo[facility][ship_type]
            probability = [tanker[fuel]['fraction_of_total'] for fuel in tanker]    
            fuel_type = random_generator.choice(list(tanker.keys()), p = probability)
            return fuel_type

In [6]:
# these pairs need to be used together for "get_fuel_type" (but don't yet have error-checks in place):
# - "WA_in_yaml" and "destination"
# - "WA_out_yaml" and "origin"

# Need to add a catch for erroneous cases where origin-destination in AIS analysis
# pairs vessel-type and facility to null values in the DOE transfer data. 
# For these cases (which shouldn't happen but might), we will use the generic US fuel allocations

# Initialize PCG-64 random number generator
random_generator = numpy.random.default_rng(random_seed)

if origin in CAD_origin_destination:
    if origin == 'Westridge Marine Terminal':
        fuel_type = get_fuel_type(CAD_yaml, origin)
    else:
        if destination in US_origin_destination:
            # we have better information on WA fuel transfers, so I'm prioritizing this information source
            fuel_type = get_fuel_type(WA_in_yaml, destination)
        else:
            fuel_type = get_fuel_type(CAD_yaml, origin)
elif origin in US_origin_destination:  
    fuel_type = get_fuel_type(WA_out_yaml, origin)
elif destination in US_origin_destination:
    fuel_type = get_fuel_type(WA_in_yaml, destination)
elif destination in CAD_origin_destination:
    fuel_type = get_fuel_type(CAD_yaml, destination)
elif origin == 'US':
    fuel_type = get_fuel_type(US_yaml, origin)
elif origin == 'Canada':
    fuel_type = get_fuel_type(US_yaml, origin)
else:
    # this is the error-check allocation for the (hopefully no) cases in which a ship track 
    # wasn't allocated either origin or destination
    fuel_type = random_generator.choice(['diesel','akns'], p = [.5, .5])